In [2]:
pip install sqlalchemy pyodbc

Note: you may need to restart the kernel to use updated packages.


Import library

In [4]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine, text  


Load data

In [5]:
# SQL Server connection details
server = 'MSI'  # Change this to your server name if needed
database = 'Test'  # Replace with your database name
driver = 'ODBC Driver 17 for SQL Server'  # Ensure this matches your installed SQL Server driver

# Create a connection string
connection_string = f"mssql+pyodbc://@{server}/{database}?driver={driver}&trusted_connection=yes"

# Create the engine
engine = create_engine(connection_string)

# Define table names
fact_customer_table = 'dbo.Fact_Customer'
dim_location_table = 'dbo.Dim_Location'
dim_date_table = 'dbo.DimDate'
dim_resell_store_table = 'dbo.Dim_ResellStore'
dim_person_table = 'dbo.Dim_Person'

# Load datasets from SQL Server
try:
    # Connect and read data from SQL Server
    with engine.connect() as connection:
        fact_customer_data = pd.read_sql(text(f"SELECT * FROM {fact_customer_table}"), connection)
        dim_location_data = pd.read_sql(text(f"SELECT * FROM {dim_location_table}"), connection)
        dim_date_data = pd.read_sql(text(f"SELECT * FROM {dim_date_table}"), connection)
        dim_resell_store_data = pd.read_sql(text(f"SELECT * FROM {dim_resell_store_table}"), connection)
        dim_person_data = pd.read_sql(text(f"SELECT * FROM {dim_person_table}"), connection)
    
    # Print sample data for verification
    print("Fact Customer Data:")
    print(fact_customer_data.head())
    
    print("Dim Location Data:")
    print(dim_location_data.head())

except Exception as e:
    print(f"Connection or query error: {e}")


Fact Customer Data:
   CustomerID  LocationID   DateKey  PersonKey  StoreKey  Demo Event  \
0       11000          25  03102013    11238.0       NaN         0.0   
1       11000          25  20062013    11238.0       NaN         0.0   
2       11000          25  21062011    11238.0       NaN         0.0   
3       11001         532  12052014     3161.0       NaN         0.0   
4       11001         532  17062011     3161.0       NaN         0.0   

   Magazine Advertisement  Manufacturer  On Promotion  Price  Quality  Review  \
0                     0.0           0.0           0.0    1.0      0.0     0.0   
1                     0.0           0.0           1.0    1.0      0.0     0.0   
2                     0.0           0.0           0.0    0.0      0.0     0.0   
3                     0.0           0.0           0.0    1.0      0.0     0.0   
4                     0.0           0.0           0.0    0.0      0.0     0.0   

   Sponsorship  Television Advertisement  Other  Unknown Rea

In [6]:
fact_customer_data['DateKey'] = pd.to_datetime(fact_customer_data['DateKey'], format='%d%m%Y', errors='coerce')

In [7]:
fact_customer_data = fact_customer_data[fact_customer_data['TotalMoneySpentByDay']>0].reset_index(drop=True)

In [8]:
fact_customer_data.head()

,CustomerID,LocationID,DateKey,PersonKey,StoreKey,Demo Event,Magazine Advertisement,Manufacturer,On Promotion,Price,Quality,Review,Sponsorship,Television Advertisement,Other,Unknown Reason,TotalMoneySpentByDay,TotalOrderCount,ModifiedDate
0,11000,25,2013-10-03,11238.0,NaN,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2770.2682,1.0,2014-09-12 11:15:07.263
1,11000,25,2013-06-20,11238.0,NaN,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,5175.7538,2.0,2014-09-12 11:15:07.263
2,11000,25,2011-06-21,11238.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3756.9890,1.0,2011-06-21 00:00:00.000
3,11001,532,2014-05-12,3161.0,NaN,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,650.8008,1.0,2014-09-12 11:15:07.263
4,11001,532,2011-06-17,3161.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3729.3640,1.0,2011-06-17 00:00:00.000


Preparing data for clustering, using elbow method to find optimal K

Performing Clustering with optimal K and display the result with the distance from zero metric to defind the label for each cluster. Since the bigger the distance from zerom, the larger RFM score is. So the cluster with higher distance from zero will be the more valuable customer groups

Yearly Clustering and summarization

In [34]:
# Convert DateKey to datetime format
fact_customer_data['DateKey'] = pd.to_datetime(fact_customer_data['DateKey'], format='%d%m%Y', errors='coerce')

# Define yearly periods
start_year = 2011
end_year = 2014

# Store yearly RFM results
yearly_rfm_data = []

# Loop over each year to calculate yearly RFM values
for year in range(start_year, end_year + 1):
    # Define the start and end of the period for each year
    period_start = pd.Timestamp(f'{year}-01-01')
    period_end = pd.Timestamp(f'{year}-12-31') if year < end_year else pd.Timestamp('2014-06-30')

    # Filter relevant data for the current year
    yearly_data = fact_customer_data[(fact_customer_data['DateKey'] >= period_start) & 
                                     (fact_customer_data['DateKey'] <= period_end)]
    
    if not yearly_data.empty:
        # Calculate RFM values for this year
        recency = yearly_data.groupby('CustomerID')['DateKey'].max().apply(lambda x: (period_end - x).days)
        frequency = yearly_data.groupby('CustomerID')['TotalOrderCount'].sum()
        monetary = yearly_data.groupby('CustomerID')['TotalMoneySpentByDay'].sum()

        # Create a DataFrame with RFM values for the year
        rfm_year = pd.DataFrame({
            'CustomerID': recency.index,
            'Recency': recency,
            'Frequency': frequency,
            'Monetary': monetary,
            'Year': year,
            'Period Start': period_start,
            'Period End': period_end
        }).reset_index(drop=True)

        # Add this year's RFM data to the list
        yearly_rfm_data.append(rfm_year)

# Combine all yearly RFM data into one DataFrame
rfm_full = pd.concat(yearly_rfm_data).reset_index(drop=True)

# Define a function to assign scores based on quantiles for Recency, Frequency, and Monetary
def assign_score(df, column):
    quantiles = df[column].quantile([0.2, 0.4, 0.6, 0.8]).drop_duplicates().values
    bins = [-float('inf')] + list(quantiles) + [float('inf')]
    labels = list(range(1, len(bins)))
    df[column + '_Score'] = pd.cut(df[column], bins=bins, labels=labels, include_lowest=True)
    return df

# Apply scores for RFM values
rfm_full = assign_score(rfm_full, 'Recency')
rfm_full = assign_score(rfm_full, 'Frequency')
rfm_full = assign_score(rfm_full, 'Monetary')

# Calculate the composite RFM score
rfm_full['RFM_Score'] = rfm_full['Recency_Score'].astype(int) + rfm_full['Frequency_Score'].astype(int) + rfm_full['Monetary_Score'].astype(int)

# Calculate the total number of customers in each group by year
summary = rfm_full.groupby(['Year', 'RFM_Score']).agg({
    'Recency': 'mean',
    'Frequency': 'mean',
    'Monetary': 'mean',
    'CustomerID': 'count'
}).rename(columns={'CustomerID': 'Customer Count'}).reset_index()

print(summary)



    Year  RFM_Score     Recency  Frequency       Monetary  Customer Count
0   2011          5   22.175000   1.000000     742.196495              40
1   2011          6   63.678571   1.000000     695.931893              28
2   2011          7   50.578313   1.048193    2400.466359              83
3   2011          8   40.237918   1.721190    7210.622167             269
4   2011          9   72.016393   1.849180   20823.137708             305
5   2011         10  100.086142   1.876404   19831.833449             267
6   2011         11  148.448430   1.789238   11841.755843             223
7   2011         12  186.424581   2.000000    7907.976800             179
8   2012          3    0.000000   1.000000      25.386800               1
9   2012          4   62.000000   1.000000      23.177300               1
10  2012          5   22.811321   1.009434     932.196987             106
11  2012          6   32.673913   1.019022    1934.220432             368
12  2012          7   66.594156   1.09

In [44]:
print(rfm_full[['CustomerID', 'Recency_Score', 'Frequency_Score', 'Monetary_Score']].query('Recency_Score == 0 or Frequency_Score == 0 or Monetary_Score == 0'))


Empty DataFrame
Columns: [CustomerID, Recency_Score, Frequency_Score, Monetary_Score]
Index: []


In [45]:
# Define the list of RFM scores for each segment
rfm_groups = {
    'Champions': [555, 554, 544, 545, 454, 455, 445],
    'Loyal Customers': [543, 444,435, 355,354,345,344,335],
    'Potential Loyalist': [553, 551,552,541,542, 533, 532,531,452,451,442,441,431,453,433,432,423,353,352,351,342,341,333,323],
    'Recent Customers': [512, 511, 422, 421, 412, 411, 311],
    'Promising': [525, 524, 523, 522, 521, 515, 514, 513, 425, 424, 413, 414, 415, 315, 314, 313],
    'Customers Needing Attention': [535, 534, 443, 434, 343, 334, 325, 324],
    'About To Sleep': [331, 321, 312, 221, 213],
    'At Risk': [255, 254, 245, 244, 253, 252, 243, 242, 235, 234, 225, 224, 153, 152, 145, 143, 142, 135, 134, 133, 125, 124],
    "Can't Lose Them": [155, 154, 144, 214, 215, 115, 114, 113],
    'Hibernating': [332, 322, 231, 241, 251, 233, 232, 223, 222, 132, 123, 122, 212, 211],
    'Lost': [111, 112, 121, 131, 141, 151]
}

# Function to calculate the RFM score and determine the segment
def rfm_segment(rfm):
    # Calculate the RFM score
    rfm_score = rfm['Recency_Score'] * 100 + rfm['Frequency_Score'] * 10 + rfm['Monetary_Score']
    
    # Check which segment the RFM score belongs to in rfm_groups
    for group, scores in rfm_groups.items():
        if rfm_score in scores:
            return group
    return 'Unknown'  # If no matching segment is found

# Apply the function to assign a segment
rfm_full['Segment'] = rfm_full.apply(rfm_segment, axis=1)

# Group by segments and calculate the mean of Recency, Frequency, and Monetary, as well as count of customers
segment_summary = rfm_full.groupby(['Year','Segment']).agg({
    'Recency': 'mean',
    'Frequency': 'mean',
    'Monetary': 'mean',
    'CustomerID': 'count'
}).rename(columns={'CustomerID': 'Customer Count'}).reset_index()

# Display the result
print(segment_summary)


    Year                      Segment     Recency  Frequency       Monetary  \
0   2011               About To Sleep   58.038462   1.000000     748.732608   
1   2011                      At Risk   39.426230   2.077283   18656.806052   
2   2011              Can't Lose Them   32.488372   1.000000    2932.796936   
3   2011  Customers Needing Attention   93.578947   2.000000   21945.938834   
4   2011                  Hibernating   61.000000   1.666667     900.373000   
5   2011              Loyal Customers   91.000000   4.000000  528278.331300   
6   2011           Potential Loyalist   91.000000   2.000000    1718.585100   
7   2011                    Promising  153.271304   1.606957    6019.962105   
8   2011             Recent Customers  132.000000   1.000000      27.762400   
9   2012               About To Sleep   57.848214   1.000000     963.141900   
10  2012                      At Risk   27.270758   3.357401   98723.634928   
11  2012              Can't Lose Them   34.437500   

In [46]:
from tabulate import tabulate

# Print RFM Segment Summary in tabular format
print(tabulate(segment_summary, headers='keys', tablefmt='grid'))


+----+--------+-----------------------------+-----------+-------------+-------------+------------------+
|    |   Year | Segment                     |   Recency |   Frequency |    Monetary |   Customer Count |
+====+========+=============================+===========+=============+=============+==================+
|  0 |   2011 | About To Sleep              |   58.0385 |    1        |    748.733  |               26 |
+----+--------+-----------------------------+-----------+-------------+-------------+------------------+
|  1 |   2011 | At Risk                     |   39.4262 |    2.07728  |  18656.8    |              427 |
+----+--------+-----------------------------+-----------+-------------+-------------+------------------+
|  2 |   2011 | Can't Lose Them             |   32.4884 |    1        |   2932.8    |              129 |
+----+--------+-----------------------------+-----------+-------------+-------------+------------------+
|  3 |   2011 | Customers Needing Attention |   93.5789

In [33]:
# import seaborn as sns
# import matplotlib.pyplot as plt

# # Plot count of customers in each segment
# plt.figure(figsize=(12, 6))
# sns.barplot(x='Segment', y='Customer Count', data=segment_summary, palette='coolwarm')
# plt.title('Customer Segmentation Based on RFM')
# plt.xticks(rotation=45)
# plt.show()


In [47]:
# Apply the function to assign a segment
rfm_full['Segment'] = rfm_full.apply(rfm_segment, axis=1)

# # Create a pivot table to display the customer and their respective segment by year
# pivot_table = rfm_full.pivot_table(index=['CustomerID', 'Year'], columns='Segment', 
#                                   values='Recency', aggfunc='first', fill_value='No Segment')

# # Reset the index to make it more readable
# pivot_table = pivot_table.reset_index()

# # Display the pivot table
# print(pivot_table)

customer_segment_by_year = rfm_full[['CustomerID', 'Segment', 'Year']].drop_duplicates()



print(customer_segment_by_year)


       CustomerID    Segment  Year
0           11000  Promising  2011
1           11001  Promising  2011
2           11002  Promising  2011
3           11003  Promising  2011
4           11004  Promising  2011
...           ...        ...   ...
25780       30114  Promising  2014
25781       30115  Promising  2014
25782       30116  Promising  2014
25783       30117  Promising  2014
25784       30118    At Risk  2014

[25785 rows x 3 columns]
